# NY flights data analysis

## Libraries

In [1]:
import pandas as pd
import json
import numpy as np 

## Data source
https://github.com/hadley/nycflights13

## Load data

In [2]:
airlines = pd.read_csv('Dane\\Dane\\airlines.csv')
airlines.head()

,carrier,name
0,9E,Endeavor Air Inc.
1,AA,American Airlines Inc.
2,AS,Alaska Airlines Inc.
3,B6,JetBlue Airways
4,DL,Delta Air Lines Inc.


In [3]:
weather = pd.read_csv(r'Dane\Dane\weather.csv', sep=';')
weather.head()

,origin_code,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,EWR,2013,1,1,1,39.02,26.06,59.37,270.0,10.35702,0.0,0.0,1012.0,10.0,2013-01-01T06:00:00Z
1,EWR,2013,1,1,2,39.02,26.96,61.63,250.0,8.05546,0.0,0.0,1012.3,10.0,2013-01-01T07:00:00Z
2,EWR,2013,1,1,3,39.02,28.04,64.43,240.0,11.50780,0.0,0.0,1012.5,10.0,2013-01-01T08:00:00Z
3,EWR,2013,1,1,4,39.92,28.04,62.21,250.0,12.65858,0.0,0.0,1012.2,10.0,2013-01-01T09:00:00Z
4,EWR,2013,1,1,5,39.02,28.04,64.43,260.0,12.65858,0.0,0.0,1011.9,10.0,2013-01-01T10:00:00Z


In [4]:
flights = pd.read_excel('Dane\\Dane\\flights.xlsx')
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00


In [5]:
planes = pd.read_csv('Dane\\Dane\\planes.txt', sep='\t')
planes.head()

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan


In [6]:
with open('Dane\\Dane\\airports.json') as f:
    airports_json = json.load(f)

airports = pd.DataFrame(airports_json)

In [7]:
airports = pd.DataFrame(airports_json)
airports.head()

,faa,name,lat,lon,alt,tz,dst,tzone
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
3,06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
4,09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York


## Top 10 airplanes with longest time in the air analysis

In [8]:
# Fill in missing data
# Airtime - average air time for all flights rounded to the nearest whole number from the entire month
flights['air_time'] = flights['air_time'].fillna(np.round(flights['air_time'].mean()))
# Plane table year - median of the year of production
planes['year'] = planes['year'].fillna(planes['year'].median())

### Manufacturer and model

In [9]:
# Include only flights with tailnum
top_10_air_time = flights[(flights['day'] <= 15) & (~flights['tailnum'].isna())].reset_index(drop=True)\
                            .loc[:, ['tailnum', 'air_time']]\
                            .merge(planes.loc[:, ['tailnum', 'manufacturer', 'model']], on='tailnum', how='left')\
                            .groupby(['manufacturer', 'model'])\
                            .agg({'air_time' : 'sum'}).reset_index()\
                            .sort_values('air_time', ascending=False).reset_index(drop=True)[:10]
top_10_air_time

,manufacturer,model,air_time
0,AIRBUS,A320-232,250071.0
1,BOEING,737-824,123399.0
2,AIRBUS INDUSTRIE,A320-232,114757.0
3,EMBRAER,EMB-145LR,101066.0
4,BOEING,757-222,79755.0
5,EMBRAER,ERJ 190-100 IGW,73541.0
6,BOEING,737-832,72894.0
7,BOEING,737-7H4,62482.0
8,EMBRAER,EMB-145XR,60970.0
9,AIRBUS,A320-214,60910.0


### Type and year of production of the oldest version

In [10]:
# First type and first year of production of top 10 planes with longest time in the air 
planes.merge(top_10_air_time, on=['manufacturer', 'model'])\
        .groupby(['manufacturer', 'model'])\
        .agg({'year' : 'min', 
              'type' : 'first'})\
        .reset_index()

,manufacturer,model,year,type
0,AIRBUS,A320-214,2001.0,Fixed wing multi engine
1,AIRBUS,A320-232,2001.0,Fixed wing multi engine
2,AIRBUS INDUSTRIE,A320-232,1993.0,Fixed wing multi engine
3,BOEING,737-7H4,1997.0,Fixed wing multi engine
4,BOEING,737-824,1998.0,Fixed wing multi engine
5,BOEING,737-832,1998.0,Fixed wing multi engine
6,BOEING,757-222,1989.0,Fixed wing multi engine
7,EMBRAER,EMB-145LR,1998.0,Fixed wing multi engine
8,EMBRAER,EMB-145XR,2002.0,Fixed wing multi engine
9,EMBRAER,ERJ 190-100 IGW,2001.0,Fixed wing multi engine


## Carrier flights analysis

### Highest number of not deleyed flights

In [11]:
# size() vs count() ; size() includes NaN
# All flights number
flights_count = flights.groupby('carrier')\
                       .size()\
                       .to_frame().reset_index()\
                       .rename({0 : 'flights_count'}, axis='columns')
flights_count

,carrier,flights_count
0,9E,1573
1,AA,2794
2,AS,62
3,B6,4427
4,DL,3690
5,EV,4171
6,F9,59
7,FL,328
8,HA,31
9,MQ,2271


In [12]:
# Not delayed flights number
not_del_count = flights.fillna({'dep_delay' : 0})\
                       .query('dep_delay <= 0')\
                       .groupby('carrier')\
                       .agg({'year' : 'count'}).reset_index()\
                       .rename({'year' : 'not_del_count'}, axis='columns')
not_del_count

,carrier,not_del_count
0,9E,999
1,AA,1890
2,AS,39
3,B6,2693
4,DL,2892
5,EV,2119
6,F9,45
7,FL,252
8,HA,20
9,MQ,1708


In [13]:
# Fill na values
flights_with_not_del = flights_count.merge(not_del_count, on='carrier', how='left').fillna(0)

In [14]:
# Not delayed flights percentage
flights_with_not_del['not_del_pr'] = flights_with_not_del['not_del_count']/flights_with_not_del['flights_count']

In [15]:
# Sort by not delayed flights percentage
flights_with_not_del.sort_values('not_del_pr', ascending=False).reset_index(drop=True)

,carrier,flights_count,not_del_count,not_del_pr
0,DL,3690,2892.0,0.783740
1,US,1602,1253.0,0.782147
2,FL,328,252.0,0.768293
3,F9,59,45.0,0.762712
4,MQ,2271,1708.0,0.752092
5,VX,316,227.0,0.718354
6,AA,2794,1890.0,0.676450
7,YV,46,31.0,0.673913
8,HA,31,20.0,0.645161
9,9E,1573,999.0,0.635092


### Highest number of delays

In [16]:
# Fill in delay missing data with 0
flights['dep_delay'] = flights['dep_delay'].fillna(0) 

In [17]:
# Count flights by carrier and origin
f_count = flights.groupby(['carrier', 'origin'])\
                 .agg({'year' : 'count'}).reset_index()\
                 .rename({'year' : 'f_count'}, axis='columns')
f_count

,carrier,origin,f_count
0,9E,EWR,82
1,9E,JFK,1419
2,9E,LGA,72
3,AA,EWR,298
4,AA,JFK,1236
5,AA,LGA,1260
6,AS,EWR,62
7,B6,EWR,573
8,B6,JFK,3327
9,B6,LGA,527


In [18]:
# Statistics for delayed flights 
tmp = flights[flights['dep_delay'] > 0]\
            .groupby(['carrier', 'origin'])\
            .agg({'day' : 'count',
                  'dep_delay' : ['mean', 
                                 'max', 
                                 np.median]})
tmp

C:\Users\Marcin\AppData\Local\Temp\ipykernel_35860\1864549728.py:4: FutureWarning: The provided callable <function median at 0x000001F172FF5E40> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  .agg({'day' : 'count',


day   dep_delay               
               count        mean     max median
carrier origin                                 
9E      EWR       18   77.000000   265.0   57.5
        JFK      539   49.936920   360.0   24.0
        LGA       17   88.647059   190.0  108.0
AA      EWR       82   50.756098   285.0   27.0
        JFK      445   29.773034   337.0   12.0
        LGA      377   26.652520   210.0   12.0
AS      EWR       23   31.869565   222.0   13.0
B6      EWR      199   42.211055   502.0   18.0
        JFK     1305   27.698084   315.0   15.0
        LGA      230   38.765217   366.0   17.0
DL      EWR       61   43.852459   262.0   20.0
        JFK      337   30.991098   599.0   13.0
        LGA      400   32.810000   478.0   16.0
EV      EWR     1930   51.553886   379.0   34.0
        JFK       31   54.967742   266.0   31.0
        LGA       91   52.087912   275.0   30.0
F9      LGA       14   55.571429   248.0   11.0
FL      LGA       76   27.565789   210.0   12.0
HA      JFK       11  159.272727  1301.0   20.0
MQ      EWR       58   60.517241  1126.0   27.5
        JFK      164   43.682927   853.0   21.0
        LGA      341   38.093842   220.0   26.0
OO      LGA        1   67.000000    67.0   67.0
UA      EWR     1726   21.904403   334.0    9.0
        JFK       94   21.808511   293.0    8.0
        LGA      250   29.900000   385.0    9.5
US      EWR       68   28.029412   214.0    9.5
        JFK       79   21.886076   164.0    7.0
        LGA      202   27.306931   336.0   17.0
VX      JFK       89   14.741573   246.0    7.0
WN      EWR      210   29.342857   256.0   13.0
        LGA      179   26.217877   259.0   10.0
YV      LGA       15   50.333333   238.0   37.0

In [19]:
# Drop multiindex for columns
tmp.columns = tmp.columns.droplevel()
tmp

count        mean     max  median
carrier origin                                   
9E      EWR        18   77.000000   265.0    57.5
        JFK       539   49.936920   360.0    24.0
        LGA        17   88.647059   190.0   108.0
AA      EWR        82   50.756098   285.0    27.0
        JFK       445   29.773034   337.0    12.0
        LGA       377   26.652520   210.0    12.0
AS      EWR        23   31.869565   222.0    13.0
B6      EWR       199   42.211055   502.0    18.0
        JFK      1305   27.698084   315.0    15.0
        LGA       230   38.765217   366.0    17.0
DL      EWR        61   43.852459   262.0    20.0
        JFK       337   30.991098   599.0    13.0
        LGA       400   32.810000   478.0    16.0
EV      EWR      1930   51.553886   379.0    34.0
        JFK        31   54.967742   266.0    31.0
        LGA        91   52.087912   275.0    30.0
F9      LGA        14   55.571429   248.0    11.0
FL      LGA        76   27.565789   210.0    12.0
HA      JFK        11  159.272727  1301.0    20.0
MQ      EWR        58   60.517241  1126.0    27.5
        JFK       164   43.682927   853.0    21.0
        LGA       341   38.093842   220.0    26.0
OO      LGA         1   67.000000    67.0    67.0
UA      EWR      1726   21.904403   334.0     9.0
        JFK        94   21.808511   293.0     8.0
        LGA       250   29.900000   385.0     9.5
US      EWR        68   28.029412   214.0     9.5
        JFK        79   21.886076   164.0     7.0
        LGA       202   27.306931   336.0    17.0
VX      JFK        89   14.741573   246.0     7.0
WN      EWR       210   29.342857   256.0    13.0
        LGA       179   26.217877   259.0    10.0
YV      LGA        15   50.333333   238.0    37.0

In [20]:
# Rename columns
f_count_del = tmp.reset_index()\
                .rename({'count' : 'd_count', 
                         'mean' : 'd_mean', 
                         'median' : 'd_median', 
                         'max' : 'd_max'}, axis='columns')
f_count_del

,carrier,origin,d_count,d_mean,d_max,d_median
0,9E,EWR,18,77.000000,265.0,57.5
1,9E,JFK,539,49.936920,360.0,24.0
2,9E,LGA,17,88.647059,190.0,108.0
3,AA,EWR,82,50.756098,285.0,27.0
4,AA,JFK,445,29.773034,337.0,12.0
5,AA,LGA,377,26.652520,210.0,12.0
6,AS,EWR,23,31.869565,222.0,13.0
7,B6,EWR,199,42.211055,502.0,18.0
8,B6,JFK,1305,27.698084,315.0,15.0
9,B6,LGA,230,38.765217,366.0,17.0


In [21]:
# Merge f_count with f_count_del
flights_with_del = f_count.merge(f_count_del, on=['carrier', 'origin'], how='left').fillna(0)

In [22]:
# Calculate percentage of delayed flights
flights_with_del['pr'] = flights_with_del['d_count']/flights_with_del['f_count']

In [23]:
# Mark carrier and origin with more than 30% of delayed flights
flights_with_del['del_flag'] = flights_with_del['pr'].apply(lambda x: int(x>0.3))
flights_with_del

,carrier,origin,f_count,d_count,d_mean,d_max,d_median,pr,del_flag
0,9E,EWR,82,18,77.000000,265.0,57.5,0.219512,0
1,9E,JFK,1419,539,49.936920,360.0,24.0,0.379845,1
2,9E,LGA,72,17,88.647059,190.0,108.0,0.236111,0
3,AA,EWR,298,82,50.756098,285.0,27.0,0.275168,0
4,AA,JFK,1236,445,29.773034,337.0,12.0,0.360032,1
5,AA,LGA,1260,377,26.652520,210.0,12.0,0.299206,0
6,AS,EWR,62,23,31.869565,222.0,13.0,0.370968,1
7,B6,EWR,573,199,42.211055,502.0,18.0,0.347295,1
8,B6,JFK,3327,1305,27.698084,315.0,15.0,0.392245,1
9,B6,LGA,527,230,38.765217,366.0,17.0,0.436433,1


In [24]:
# Set index to carrier
airlines.index = airlines['carrier']

In [25]:
# Create dictionary with carrier short and full name 
carriers_dict = airlines['name'].to_dict()
carriers_dict

{'9E': 'Endeavor Air Inc.',
 'AA': 'American Airlines Inc.',
 'AS': 'Alaska Airlines Inc.',
 'B6': 'JetBlue Airways',
 'DL': 'Delta Air Lines Inc.',
 'EV': 'ExpressJet Airlines Inc.',
 'F9': 'Frontier Airlines Inc.',
 'FL': 'AirTran Airways Corporation',
 'HA': 'Hawaiian Airlines Inc.',
 'MQ': 'Envoy Air',
 'OO': 'SkyWest Airlines Inc.',
 'UA': 'United Air Lines Inc.',
 'US': 'US Airways Inc.',
 'VX': 'Virgin America',
 'WN': 'Southwest Airlines Co.',
 'YV': 'Mesa Airlines Inc.'}

In [26]:
# Map short name of carriers with its full name 
flights_with_del['carrier_full_name'] = flights_with_del['carrier'].map(carriers_dict)
flights_with_del

,carrier,origin,f_count,d_count,d_mean,d_max,d_median,pr,del_flag,carrier_full_name
0,9E,EWR,82,18,77.000000,265.0,57.5,0.219512,0,Endeavor Air Inc.
1,9E,JFK,1419,539,49.936920,360.0,24.0,0.379845,1,Endeavor Air Inc.
2,9E,LGA,72,17,88.647059,190.0,108.0,0.236111,0,Endeavor Air Inc.
3,AA,EWR,298,82,50.756098,285.0,27.0,0.275168,0,American Airlines Inc.
4,AA,JFK,1236,445,29.773034,337.0,12.0,0.360032,1,American Airlines Inc.
5,AA,LGA,1260,377,26.652520,210.0,12.0,0.299206,0,American Airlines Inc.
6,AS,EWR,62,23,31.869565,222.0,13.0,0.370968,1,Alaska Airlines Inc.
7,B6,EWR,573,199,42.211055,502.0,18.0,0.347295,1,JetBlue Airways
8,B6,JFK,3327,1305,27.698084,315.0,15.0,0.392245,1,JetBlue Airways
9,B6,LGA,527,230,38.765217,366.0,17.0,0.436433,1,JetBlue Airways


In [27]:
# Sort final table
flights_with_del.sort_values('d_count', ascending=False)

,carrier,origin,f_count,d_count,d_mean,d_max,d_median,pr,del_flag,carrier_full_name
13,EV,EWR,3838,1930,51.553886,379.0,34.0,0.502866,1,ExpressJet Airlines Inc.
23,UA,EWR,3657,1726,21.904403,334.0,9.0,0.471972,1,United Air Lines Inc.
8,B6,JFK,3327,1305,27.698084,315.0,15.0,0.392245,1,JetBlue Airways
1,9E,JFK,1419,539,49.936920,360.0,24.0,0.379845,1,Endeavor Air Inc.
4,AA,JFK,1236,445,29.773034,337.0,12.0,0.360032,1,American Airlines Inc.
12,DL,LGA,1889,400,32.810000,478.0,16.0,0.211752,0,Delta Air Lines Inc.
5,AA,LGA,1260,377,26.652520,210.0,12.0,0.299206,0,American Airlines Inc.
21,MQ,LGA,1470,341,38.093842,220.0,26.0,0.231973,0,Envoy Air
11,DL,JFK,1522,337,30.991098,599.0,13.0,0.221419,0,Delta Air Lines Inc.
25,UA,LGA,600,250,29.900000,385.0,9.5,0.416667,1,United Air Lines Inc.


### Highest number of flights outside of NY

In [28]:
# Get area from tzone
airports['area'] = airports['tzone'].apply(lambda x: x.split('/')[1].replace('_', ' ') if x else x)

In [29]:
# Different_tzone_flights_count - count of flights outside the start area by carrier and origin
# No timezone information - assume flight outside the start area
# Show only combinations with the flights count > 100
flights_outside_NY = flights.loc[:, ['carrier', 'origin', 'dest']]\
                            .merge(airports.loc[:, ['faa', 'area']], left_on='origin', right_on='faa', how='left')\
                            .rename({'area' : 'dep_area'}, axis='columns')\
                            .merge(airports.loc[:, ['faa', 'area']], left_on='dest', right_on='faa', how='left')\
                            .rename({'area' : 'arr_area'}, axis='columns')\
                            .drop(['faa_x', 'faa_y'], axis='columns')\
                            .query('dep_area != arr_area').reset_index(drop=True)\
                            .groupby(['carrier', 'origin'])\
                            .agg({'dest' : 'count'}).reset_index()\
                            .query('dest>=100').reset_index(drop=True)\
                            .sort_values('dest', ascending=False).reset_index(drop=True)\
                            .rename({'dest' : 'different_tzone_flights_count'}, axis='columns')

flights_outside_NY

,carrier,origin,different_tzone_flights_count
0,UA,EWR,2074
1,B6,JFK,1201
2,AA,LGA,929
3,DL,JFK,923
4,EV,EWR,902
5,AA,JFK,767
6,UA,LGA,573
7,WN,LGA,452
8,WN,EWR,442
9,DL,LGA,401


### Most popular destination and maximum number of passengers it could carry on

In [30]:
# Most popular destination and maximum number of passengers it could carry on (the same timezone enabled)
most_pop_dest_max_passengers = flights_outside_NY.merge(
                                    flights.loc[:, ['carrier', 'origin', 'dest', 'tailnum']]\
                                        .merge(airports.loc[:, ['faa', 'area']], left_on='dest', right_on='faa', how='left')\
                                        .rename({'area' : 'arr_area'}, axis='columns')\
                                        .merge(planes.loc[:, ['tailnum', 'seats']], on='tailnum', how='left')\
                                        .groupby(['carrier', 'origin', 'arr_area'])\
                                        .agg({'faa' : 'count', 
                                              'seats' : 'sum'}).reset_index()\
                                        .sort_values(['carrier', 'origin', 'faa']).reset_index(drop=True)\
                                        .groupby(['carrier', 'origin'])\
                                        .agg({'arr_area' : 'last', 
                                              'faa' : 'last', 
                                              'seats' : 'last'}).reset_index()\
                                        .rename({'faa' : 'most_pop_arr_area_flights_count', 
                                                 'seats' : 'most_pop_arr_area_seats_sum', 
                                                 'arr_area' : 'most_pop_arr_area'}, axis='columns'), 
                                        on=['carrier', 'origin']
                            )

most_pop_dest_max_passengers

,carrier,origin,different_tzone_flights_count,most_pop_arr_area,most_pop_arr_area_flights_count,most_pop_arr_area_seats_sum
0,UA,EWR,2074,New York,1583,265675.0
1,B6,JFK,1201,New York,2126,247505.0
2,AA,LGA,929,Chicago,929,37891.0
3,DL,JFK,923,New York,599,103225.0
4,EV,EWR,902,New York,2936,165930.0
5,AA,JFK,767,Los Angeles,488,89984.0
6,UA,LGA,573,Chicago,433,74426.0
7,WN,LGA,452,Chicago,391,54944.0
8,WN,EWR,442,Chicago,319,44961.0
9,DL,LGA,401,New York,1488,243467.0


In [31]:
# Map full names of origin airports
most_pop_dest_max_passengers['origin_full'] = most_pop_dest_max_passengers['origin']\
                                                .map({'EWR' : 'Newark Liberty International Airport', 
                                                      'JFK' : 'John F. Kennedy International Airport', 
                                                      'LGA' : 'LaGuardia Airport'})
most_pop_dest_max_passengers

,carrier,origin,different_tzone_flights_count,most_pop_arr_area,most_pop_arr_area_flights_count,most_pop_arr_area_seats_sum,origin_full
0,UA,EWR,2074,New York,1583,265675.0,Newark Liberty International Airport
1,B6,JFK,1201,New York,2126,247505.0,John F. Kennedy International Airport
2,AA,LGA,929,Chicago,929,37891.0,LaGuardia Airport
3,DL,JFK,923,New York,599,103225.0,John F. Kennedy International Airport
4,EV,EWR,902,New York,2936,165930.0,Newark Liberty International Airport
5,AA,JFK,767,Los Angeles,488,89984.0,John F. Kennedy International Airport
6,UA,LGA,573,Chicago,433,74426.0,LaGuardia Airport
7,WN,LGA,452,Chicago,391,54944.0,LaGuardia Airport
8,WN,EWR,442,Chicago,319,44961.0,Newark Liberty International Airport
9,DL,LGA,401,New York,1488,243467.0,LaGuardia Airport


In [32]:
# Sort final table
most_pop_dest_max_passengers.sort_values('most_pop_arr_area_flights_count', ascending=False)

,carrier,origin,different_tzone_flights_count,most_pop_arr_area,most_pop_arr_area_flights_count,most_pop_arr_area_seats_sum,origin_full
4,EV,EWR,902,New York,2936,165930.0,Newark Liberty International Airport
1,B6,JFK,1201,New York,2126,247505.0,John F. Kennedy International Airport
0,UA,EWR,2074,New York,1583,265675.0,Newark Liberty International Airport
9,DL,LGA,401,New York,1488,243467.0,LaGuardia Airport
13,9E,JFK,302,New York,1117,79770.0,John F. Kennedy International Airport
11,MQ,LGA,355,New York,1115,1360.0,LaGuardia Airport
2,AA,LGA,929,Chicago,929,37891.0,LaGuardia Airport
3,DL,JFK,923,New York,599,103225.0,John F. Kennedy International Airport
5,AA,JFK,767,Los Angeles,488,89984.0,John F. Kennedy International Airport
6,UA,LGA,573,Chicago,433,74426.0,LaGuardia Airport
